In [ ]:
import pprint
import shutil

import kagglehub
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

/Users/tonipenya/.pyenv/versions/emotion_detection/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download raw data


In [ ]:
path = kagglehub.dataset_download("msambare/fer2013")
path

'/Users/tonipenya/.cache/kagglehub/datasets/msambare/fer2013/versions/1'

In [ ]:
shutil.copytree(path, "data/raw", dirs_exist_ok=True)

'data/raw'

# Create dataset


In [ ]:
TRAIN_PATH = "data/raw/train"
batch_size = 64
img_size = 48
num_workers = 4
learning_rate = 1e-3
val_ratio = 0.2
seed = 42
epochs = 10

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ]
)

In [ ]:
full_dataset = datasets.ImageFolder(root=TRAIN_PATH, transform=train_transforms)

val_count = int(len(full_dataset) * val_ratio)
train_count = len(full_dataset) - val_count
train_ds, val_ds = random_split(
    full_dataset,
    [train_count, val_count],
    generator=torch.Generator().manual_seed(seed),
)

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)

val_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

In [ ]:
num_classes = len(full_dataset.classes)

f"{len(full_dataset)} samples", f"{num_classes} classes: {full_dataset.classes}"

('28709 samples',
 "7 classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']")

# Train model


In [ ]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Sequential(
            nn.Linear((img_size // 8) * (img_size // 8) * 128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        self.fc2 = nn.Linear(256, num_classes)
        self.float()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.flatten(1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
print(device)

mps


In [ ]:
model = TinyCNN(num_classes=num_classes).to(device).to(torch.float32)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for ep in range(epochs):
    model.train()
    tr_loss = 0

    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device)
        opt.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        opt.step()
        tr_loss += loss.item() * images.size(0)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            val_loss += loss.item() * images.size(0)

    print(
        f"Epoch {ep}/{epochs}"
        f" | Train Loss: {tr_loss/len(train_ds):.4f}"
        f" | Val Loss: {val_loss/len(val_ds):.4f}"
    )

/Users/tonipenya/.pyenv/versions/emotion_detection/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 0/10 | Train Loss: 1.6580 | Val Loss: 5.8080
Epoch 1/10 | Train Loss: 1.3849 | Val Loss: 4.9038
Epoch 2/10 | Train Loss: 1.2322 | Val Loss: 4.3676
Epoch 3/10 | Train Loss: 1.1160 | Val Loss: 3.8714
Epoch 4/10 | Train Loss: 1.0069 | Val Loss: 3.3066
Epoch 5/10 | Train Loss: 0.8991 | Val Loss: 2.9043
Epoch 6/10 | Train Loss: 0.7901 | Val Loss: 2.4246
Epoch 7/10 | Train Loss: 0.6806 | Val Loss: 1.9213
Epoch 8/10 | Train Loss: 0.5722 | Val Loss: 1.5904
Epoch 9/10 | Train Loss: 0.4757 | Val Loss: 1.1625


## Training metrics


In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        logits = model(images)
        y_true.append(labels)
        y_pred.append(logits.argmax(1).cpu())

y_true = torch.cat(y_true).numpy()
y_pred = torch.cat(y_pred).numpy()

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="macro", zero_division=0
)
precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="micro", zero_division=0
)

precision_class, recall_class, f1_class, _ = precision_recall_fscore_support(
    y_true, y_pred, average=None, zero_division=0
)

In [ ]:
metrics = {
    "global": {
        "accuracy": accuracy,
        "micro": {
            "precision": precision_micro,
            "recall": recall_micro,
            "f1-score": f1_micro,
        },
        "macro": {
            "precision": precision_macro,
            "recall": recall_macro,
            "f1-score": f1_macro,
        },
    },
    "per_class": {
        class_name: {
            "precision": precision_class[i],
            "recall": recall_class[i],
            "f1-score": f1_class[i],
        }
        for i, class_name in enumerate(full_dataset.classes)
    },
}

pprint.pprint(metrics)

{'global': {'accuracy': 0.9108324625566004,
            'macro': {'f1-score': 0.9080093997911245,
                      'precision': 0.915492229510482,
                      'recall': 0.9024947217554484},
            'micro': {'f1-score': 0.9108324625566004,
                      'precision': 0.9108324625566004,
                      'recall': 0.9108324625566004}},
 'per_class': {'angry': {'f1-score': np.float64(0.8837209302325582),
                         'precision': np.float64(0.9068466096115866),
                         'recall': np.float64(0.8617453862996559)},
               'disgust': {'f1-score': np.float64(0.9274074074074075),
                           'precision': np.float64(0.9842767295597484),
                           'recall': np.float64(0.876750700280112)},
               'fear': {'f1-score': np.float64(0.8565270935960592),
                        'precision': np.float64(0.8567908838928241),
                        'recall': np.float64(0.8562634656817483)},
         

## Export model


In [ ]:
torch.save(
    {"model_state": model.state_dict(), "classes": full_dataset.classes},
    "data/model.pt",
)

# Evaluate model


In [ ]:
TEST_PATH = "data/raw/test"

In [33]:
test_ds = datasets.ImageFolder(root=TEST_PATH, transform=train_transforms)
test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

In [34]:
checkpoint = torch.load("data/model.pt")
model = TinyCNN(num_classes=len(checkpoint["classes"]))
model.load_state_dict(checkpoint["model_state"])

<All keys matched successfully>

In [35]:
model.eval()
model.to(device)
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        logits = model(images)
        y_true.append(labels)
        y_pred.append(logits.argmax(1).cpu())

y_true = torch.cat(y_true).numpy()
y_pred = torch.cat(y_pred).numpy()

/Users/tonipenya/.pyenv/versions/emotion_detection/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


In [36]:
accuracy = accuracy_score(y_true, y_pred)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="macro", zero_division=0
)
precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="micro", zero_division=0
)

precision_class, recall_class, f1_class, _ = precision_recall_fscore_support(
    y_true, y_pred, average=None, zero_division=0
)

In [37]:
metrics = {
    "global": {
        "accuracy": accuracy,
        "micro": {
            "precision": precision_micro,
            "recall": recall_micro,
            "f1-score": f1_micro,
        },
        "macro": {
            "precision": precision_macro,
            "recall": recall_macro,
            "f1-score": f1_macro,
        },
    },
    "per_class": {
        class_name: {
            "precision": precision_class[i],
            "recall": recall_class[i],
            "f1-score": f1_class[i],
        }
        for i, class_name in enumerate(full_dataset.classes)
    },
}

pprint.pprint(metrics)

{'global': {'accuracy': 0.564641961549178,
            'macro': {'f1-score': 0.553377293616537,
                      'precision': 0.58170767860571,
                      'recall': 0.5389793209070641},
            'micro': {'f1-score': 0.564641961549178,
                      'precision': 0.564641961549178,
                      'recall': 0.564641961549178}},
 'per_class': {'angry': {'f1-score': np.float64(0.4668380462724936),
                         'precision': np.float64(0.45997973657548125),
                         'recall': np.float64(0.47390396659707723)},
               'disgust': {'f1-score': np.float64(0.56),
                           'precision': np.float64(0.765625),
                           'recall': np.float64(0.44144144144144143)},
               'fear': {'f1-score': np.float64(0.3848396501457726),
                        'precision': np.float64(0.3829787234042553),
                        'recall': np.float64(0.38671875)},
               'happy': {'f1-score': np.flo